# 2.2 Tuning to Avoid Key Lookups and Residual Predicates (10m)

sp_BlitzCache’s warning about Expensive Key Lookups has a few different possible solutions:

- Look for predicates on the lookup for surefire things to add to the nonclustered index
- Look for output fields to consider, but maybe not add
- Or just live with the key lookups when they’re big fields, or not a lot of rows

## Questions
- What is a key lookup?
- How do we mitigate key lookups?
    - Select fewer fields
    - Add more fields to the nonclustered index
- What is a residual predicate? Why is eliminating them so important?

## From How to Think Like the Engine


In [ ]:
CREATE INDEX IX_LastAccessDate_Id
ON dbo.Users (LastAccessDate, Id);


-- Build the query plan for this
SELECT *
FROM dbo.Users
WHERE LastAccessDate >= '2011/11/10'
AND LastAccessDate <= '2011/11/11'

Since we have a tight range, SQL will do a key lookup for every row we found:

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.01.png'>

The more rows SQL finds, the more executions SQL has to do to get the results for SELECT *:
<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.02.png'>

## What is a key lookup?

- The optimizer made a choice (for better or worse):
    - It used a NC index to make a query more efficient...
    - But it didn't have all the columns the query neededed
    > These could be missing columns in the SELECT list, or missing other predicates / JOIN conditions <br>
    > Or, it wasn't a 'covering' index
- The problem with Key lookups
    - They introduce a lot of random I/O
    - They run in a nested loops join, which turns into a problem if a lot of rows are needed

## Behind the secnes

SQL start grabbing rows from the NC Index

- CX key columns are part of every NC index - another reason why CX key column choice is important
- They get used to join one index to another
    - If the CX isn't unique, uniquifiers get involved
    - If the table is a HEAP, RIDS *(RowId's)* are used instead

In a cooking analogy, the Clustered Index is the equivalent to the Grocery Store - if we don't have the ingredients (fields) on hand (NCI), we make a trip (key lookup) to the store (CDX)


## The Catch: 1 Row At a Time

However, Key Lookups work 1 row at a time! Imagine buying a bag of chips, but making multiple trips to the store, buying 1 chip at a time...

For every row that comes out of the NC index:
1. Goes to the CDX
2. Locates the row based on the CDX key column(s)
3. Gets the additional columns needed for the query

And repeat the process for EACH row!!

## When it makes sense

If you don't have 'too many' rows (eg. under 1000 or so), they're not so bad. The more rows they need to fetch, the worse it gets

There are tipping points:

- SQL is lazy, and if it things a lot of rows will come back, it will opt to scan the CDX instead
- In rare circumstances, *index intersection* may occur:
    - This is when SQL joins two NC indexes together
    - Uses the CX key columns, which are present in both

In our case, our date range was selective. However, if we expand the date range:


In [ ]:
SELECT *
FROM dbo.Users -- WITH (INDEX = IX_LastAccessDate_Id)
WHERE LastAccessDate >= '2011/01/01'
AND LastAccessDate <= '2011/12/31'


... it doesn't make sense to do so many key lookups when we expect the query to return a lot of rows 

## Force the index

We can choose to force using the index hint, but then SQL server has to execute this key lookup each time. As we can see, it has to end up reading more pages than there are in the table:

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.03.png'>

SQL Server automatically decides when a clustered index scan is more efficient, and it's not always right about that either.

## Identifying Expensive Key Lookups

### sp_BlitzCache will warn you

About this any many other things:

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.04.png'>

Note the number of executions! Quite a lot!

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.05.png' width=500>

### 2 Options to fix Key Lookup

1. Select less fields (less likely...)
2. Add fields to the nonclustered index (especially if they're residual predicates)

### Residual Predicate

Looking back to our one-day query:

In [ ]:
SELECT *
FROM dbo.Users -- WITH (INDEX = IX_LastAccessDate_Id)
WHERE LastAccessDate >= '2011/01/01'
AND LastAccessDate <= '2011/12/31'

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.06.png' width=500>

So what happens if we add a fields to the WHERE clause...

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.07.png' width=500>

Our query is now looking for a field that doesn't exist in the index, so we have to do a key lookup to check it. This is known as a **Residual Predicate**, and will appear under the 'Predicate' section of the Key Lookup. 

<img src='C:\Users\hartleyg\Desktop\Training\sqltraining\Brent Ozar\Mastering Index Tuning\2.2 Tuning to Avoid Key Lookups and Residual Predicates\2.2.08.png' width=300>

This is different from the Seek Predicate, which occurs when the field we're searching exists in the index

### Fixing residual predicates

- Add them to the non-clustered index
- We can check during the fast nonclustered index operation
- We can cut down on the number of key lookups performed (or eliminate em)

### Why it's good to fix them

Makes your code less senstive to parameter sniffing:
- If a 'small' query runs first and gets cached, the key lookup plan may make sense
- If a 'large' query runs second, it will use the cached plan of the small query (ie. from 100 key looksup to 100k)
- With the right covering index for the query, it doesnt matter which one runs first
    - Plan shape wont change as if we can eliminate key lookups
    - May be other differences, like parallelism or join types

## Recap

- sp_BlitzCache warns about expensive key lookups
- Most easy to fix by adding columns to an index
- Really long output lists may not be worth covering
- Residual predicates? Move those fields to the index
> Note: Even adding as an INCLUDES column will be sufficient!
- If several predicate lookups are involved, it might be better to add the exact covering index from scratch

